In [7]:
import pandas as pd
import numpy as np
import datetime
import csv

from DATA225utils import make_connection, dataframe_query 

In [8]:
games_details = pd.read_csv("data/games_details.csv", low_memory = False)

games = pd.read_csv("data/games.csv", low_memory = False)

players = pd.read_csv("data/players.csv", low_memory = False)

ranking = pd.read_csv("data/ranking.csv", low_memory = False)

teams_df = pd.read_csv("data/teams.csv", low_memory = False)

In [9]:
games_details = games_details.drop_duplicates(subset = ["GAME_ID", "TEAM_ID", "PLAYER_ID"])
games = games.drop_duplicates(subset = "GAME_ID")
players = players.drop_duplicates(subset = "PLAYER_ID")
ranking = ranking.drop_duplicates(subset = ["TEAM_ID", "STANDINGSDATE"])
teams_df = teams_df.drop_duplicates(subset = "TEAM_ID")

In [10]:
games_details.drop(["NICKNAME", "START_POSITION", "COMMENT", "PLUS_MINUS"], axis = 1, inplace = True)

In [11]:
games = games[games.GAME_DATE_EST>'2004-01-01']

In [12]:
ranking = ranking[ranking.STANDINGSDATE>'2004-01-01']

In [13]:
ranking.drop("RETURNTOPLAY", axis = 1, inplace = True)

In [14]:
players.isna().sum()

PLAYER_NAME    0
TEAM_ID        0
PLAYER_ID      0
SEASON         0
dtype: int64

In [15]:
players.shape

(1769, 4)

In [8]:
df = pd.read_csv("data/all_seasons.csv")

In [9]:
def get_height(player_name):
    try:
        return list(df[df.player_name==player_name]["player_height"].drop_duplicates())[0]
    except:
        return None

def get_draft_year(player_name):
    try:
        return list(df[df.player_name==player_name]["draft_year"].drop_duplicates())[0]
    except:
        return None

In [10]:
players["HEIGHT"] = players.PLAYER_NAME.apply(lambda x: get_height(x))
players["DRAFTYEAR"] = players.PLAYER_NAME.apply(lambda x: get_draft_year(x))

In [12]:
players[players.HEIGHT.isna()]

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON,HEIGHT,DRAFTYEAR
428,Charles Brown Jr.,1610612737,1629718,2019,NaN,None
456,Michael Frazier,1610612745,1626187,2019,NaN,None
478,Christ Koumadje,1610612755,1629746,2019,NaN,None
479,Isaiah Miles,1610612755,1627776,2019,NaN,None
485,CJ Massinburg,1610612751,1629747,2019,NaN,None
...,...,...,...,...,...,...
7217,David Monds,1610612747,201776,2009,NaN,None
7223,Lanny Smith,1610612758,201831,2009,NaN,None
7224,Warren Carter,1610612752,201999,2009,NaN,None
7225,Bennet Davis,1610612751,201834,2009,NaN,None


In [14]:
players.isna().sum()

PLAYER_NAME      0
TEAM_ID          0
PLAYER_ID        0
SEASON           0
HEIGHT         347
DRAFTYEAR      347
dtype: int64

In [27]:
# from nba_api.stats.endpoints import commonplayerinfo

# # Nikola Jokić
# career = commonplayerinfo.CommonPlayerInfo('1629718') 

# # pandas data frames (optional: pip install pandas)
# career.get_data_frames()[0].columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG'],
      dtype='object')

In [28]:
from nba_api.stats.endpoints import commonplayerinfo

def getPlayerInfo(player_id):
    return commonplayerinfo.CommonPlayerInfo(player_id).get_data_frames()[0][['BIRTHDATE', 'COUNTRY', 'HEIGHT',
       'WEIGHT', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']]

In [46]:
players[['BIRTHDATE', 'COUNTRY', 'HEIGHT', \
       'WEIGHT', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']] = np.nan

In [58]:
df = getPlayerInfo(1626187)
df

,BIRTHDATE,COUNTRY,HEIGHT,WEIGHT,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,1994-03-08T00:00:00,USA,6-3,200,Undrafted,Undrafted,Undrafted,N


In [87]:
df['COUNTRY'][0]

'USA'

In [88]:
players.loc[players.PLAYER_ID==1626187,['BIRTHDATE']] = df["BIRTHDATE"][0]

In [89]:
players[players.PLAYER_ID==1626187].loc[:,['BIRTHDATE', 'COUNTRY', 'HEIGHT',\
       'WEIGHT', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']]

,BIRTHDATE,COUNTRY,HEIGHT,WEIGHT,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
456,1994-03-08T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
import time
for i in players[players.DRAFT_NUMBER.isna()].PLAYER_ID:
    try:
        df = getPlayerInfo(i)
        players.loc[players.PLAYER_ID==i, ['BIRTHDATE']] = df.BIRTHDATE[0]
        players.loc[players.PLAYER_ID==i, ['COUNTRY']] = df.COUNTRY[0]
        players.loc[players.PLAYER_ID==i, ['HEIGHT']] = df.HEIGHT[0]
        players.loc[players.PLAYER_ID==i, ['WEIGHT']] = df.WEIGHT[0]
        players.loc[players.PLAYER_ID==i, ['DRAFT_YEAR']] = df.DRAFT_YEAR[0]
        players.loc[players.PLAYER_ID==i, ['DRAFT_ROUND']] = df.DRAFT_ROUND[0]
        players.loc[players.PLAYER_ID==i, ['DRAFT_NUMBER']] = df.DRAFT_NUMBER[0]
        players.loc[players.PLAYER_ID==i, ['GREATEST_75_FLAG']] = df.GREATEST_75_FLAG[0]
        time.sleep(0.5)
    except:
        continue

In [114]:
players.isna().sum()

PLAYER_NAME           0
TEAM_ID               0
PLAYER_ID             0
SEASON                0
BIRTHDATE            23
COUNTRY              91
HEIGHT               23
WEIGHT               23
DRAFT_YEAR           23
DRAFT_ROUND         126
DRAFT_NUMBER        129
GREATEST_75_FLAG     23
dtype: int64

In [113]:
players.BIRTHDATE = pd.to_datetime(players.BIRTHDATE)

In [116]:
players.head()

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON,BIRTHDATE,COUNTRY,HEIGHT,WEIGHT,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,Royce O'Neale,1610612762,1626220,2019,1993-06-05,USA,6-6,226,Undrafted,Undrafted,Undrafted,N
1,Bojan Bogdanovic,1610612762,202711,2019,1989-04-18,Croatia,6-7,226,2011,2,31,N
2,Rudy Gobert,1610612762,203497,2019,1992-06-26,France,7-1,258,2013,1,27,N
3,Donovan Mitchell,1610612762,1628378,2019,1996-09-07,USA,6-1,215,2017,1,13,N
4,Mike Conley,1610612762,201144,2019,1987-10-11,USA,6-1,175,2007,1,4,N


In [118]:
ranking.head()

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7


In [120]:
ranking.LEAGUE_ID.value_counts()

0    207616
Name: LEAGUE_ID, dtype: int64

In [161]:
ranking[["STARTDATE"]] = np.nan

In [162]:
ranking

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,STARTDATE
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210337,1610612765,0,22013,2014-09-01,East,Detroit,82,29,53,0.354,17-24,12-29,NaN
210338,1610612738,0,22013,2014-09-01,East,Boston,82,25,57,0.305,16-25,9-32,NaN
210339,1610612753,0,22013,2014-09-01,East,Orlando,82,23,59,0.280,19-22,4-37,NaN
210340,1610612755,0,22013,2014-09-01,East,Philadelphia,82,19,63,0.232,10-31,9-32,NaN


In [164]:
for i in ranking.SEASON_ID.unique():
    ranking.loc[ranking.SEASON_ID==i, ['STARTDATE']] = min(ranking[ranking.SEASON_ID==i].STANDINGSDATE)

In [165]:
ranking

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,STARTDATE
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,2022-10-18
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,2022-10-18
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,2022-10-18
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,2022-10-18
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,2022-10-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210337,1610612765,0,22013,2014-09-01,East,Detroit,82,29,53,0.354,17-24,12-29,2013-10-29
210338,1610612738,0,22013,2014-09-01,East,Boston,82,25,57,0.305,16-25,9-32,2013-10-29
210339,1610612753,0,22013,2014-09-01,East,Orlando,82,23,59,0.280,19-22,4-37,2013-10-29
210340,1610612755,0,22013,2014-09-01,East,Philadelphia,82,19,63,0.232,10-31,9-32,2013-10-29


In [166]:
ranking.isna().sum()

TEAM_ID          0
LEAGUE_ID        0
SEASON_ID        0
STANDINGSDATE    0
CONFERENCE       0
TEAM             0
G                0
W                0
L                0
W_PCT            0
HOME_RECORD      0
ROAD_RECORD      0
STARTDATE        0
dtype: int64

In [168]:
players.to_csv("data/new_players_data.csv")

In [169]:
df = getPlayerInfo(731)

In [171]:
players

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON,BIRTHDATE,COUNTRY,HEIGHT,WEIGHT,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,Royce O'Neale,1610612762,1626220,2019,1993-06-05,USA,6-6,226,Undrafted,Undrafted,Undrafted,N
1,Bojan Bogdanovic,1610612762,202711,2019,1989-04-18,Croatia,6-7,226,2011,2,31,N
2,Rudy Gobert,1610612762,203497,2019,1992-06-26,France,7-1,258,2013,1,27,N
3,Donovan Mitchell,1610612762,1628378,2019,1996-09-07,USA,6-1,215,2017,1,13,N
4,Mike Conley,1610612762,201144,2019,1987-10-11,USA,6-1,175,2007,1,4,N
...,...,...,...,...,...,...,...,...,...,...,...,...
7223,Lanny Smith,1610612758,201831,2009,1984-10-30,None,,,Undrafted,Undrafted,Undrafted,N
7224,Warren Carter,1610612752,201999,2009,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7225,Bennet Davis,1610612751,201834,2009,1984-03-14,None,,,Undrafted,Undrafted,Undrafted,N
7226,Brian Hamilton,1610612751,201646,2009,1982-05-14,USA,,,Undrafted,Undrafted,Undrafted,N
